In [31]:
import pandas as pd
import re
import glob
import os


In [32]:
import pandas as pd
import re
import os

# ---- Учитај фајлове ----
df1 = pd.read_csv("basic_imsbs.csv") 

df2 = pd.read_csv("learning_U10_10_A1_F1.csv")  

# ---- Извуци 4 броја из имена фајла ----
def extract_numbers(filename):
    # узимамо сва 4 броја из имена: n_m_k_index
    m = re.match(r".*?_(\d+)_(\d+)_(\d+)_(\d+)\.txt\.out", filename)
    if m:
        return int(m.group(1)), int(m.group(2)), int(m.group(3)), int(m.group(4))
    else:
        return None, None, None, None

for df in [df1, df2]:
    df[['n','m','k','idx']] = df['file'].apply(lambda x: pd.Series(extract_numbers(x)))
    # kreiramo key po prva tri broja
    df['key'] = df[['n','m','k']].astype(str).agg('_'.join, axis=1)

# ---- Spoji podatke po key ----
merged = pd.merge(df1, df2, on='key', suffixes=('_1','_2'))

# ---- Agregiraj po prva tri broja (key) ----
agg = merged.groupby('key').agg({
    'time_1': 'mean',
    'quality_1': 'mean',
    'time_2': 'mean',
    'quality_2': 'mean'
}).reset_index()

# ---- Rezultat ----
print(agg)

# ---- Sačuvaj u CSV ----
agg.to_csv("merged_aggregated.csv", index=False)


         key    time_1  quality_1     time_2  quality_2
0   10_100_2  0.010758        8.6   0.014024        8.7
1   10_100_4  0.015646        6.2   0.017687        6.2
2   10_200_2  0.017391       10.3   0.019955       10.3
3   10_200_4  0.016767        6.2   0.020334        6.1
4   10_500_2  0.009166        9.5   0.039018        9.5
5   10_500_4  0.018688        6.1   0.027626        6.1
6    10_50_2  0.013251        9.1   0.011235        9.1
7    10_50_4  0.014968        6.0   0.016039        6.0
8    2_100_2  0.074699       72.8   0.189357       72.7
9    2_100_4  0.076843       61.7   0.173430       61.7
10   2_200_2  0.329666      136.9   1.172928      139.7
11   2_200_4  0.394792      125.1   1.065063      120.4
12   2_500_2  3.156137      272.1  14.869239      281.2
13   2_500_4  3.025999      299.3   9.651473      293.2
14    2_50_2  0.006316       37.7   0.009290       37.7
15    2_50_4  0.014924       30.1   0.029296       30.1
16   3_100_2  0.229338       58.5   0.518634    

In [33]:
agg[["time_1", "time_2", "quality_1", "quality_2" ]].mean()

time_1        0.769833
time_2        3.129283
quality_1    60.431250
quality_2    60.081250
dtype: float64

In [34]:
# ---- Basic baseline ----
df_basic = pd.read_csv("basic_imsbs.csv")
df_basic["source"] = "basic"


In [35]:
# ---- All learning CSVs ----
learning_files = glob.glob("learning_U*_A*_F*.csv")

dfs_learning = []
for f in learning_files:
    df = pd.read_csv(f)
    df["source"] = os.path.basename(f).replace(".csv", "")
    dfs_learning.append(df)

df_learning = pd.concat(dfs_learning, ignore_index=True)

def extract_numbers(filename):
    """
    Extract n, m, k, idx from:
    ..._n_m_k_idx.txt.out
    """
    m = re.search(r"_(\d+)_(\d+)_(\d+)_(\d+)\.txt\.out", filename)
    if m:
        return int(m.group(1)), int(m.group(2)), int(m.group(3)), int(m.group(4))
    return None, None, None, None



In [36]:
for df in [df_basic, df_learning]:
    df[["n", "m", "k", "idx"]] = df["file"].apply(
        lambda x: pd.Series(extract_numbers(x))
    )
    df["key"] = df[["n", "m", "k"]].astype(str).agg("_".join, axis=1)


In [37]:
df_all = pd.concat([df_basic, df_learning], ignore_index=True)


In [38]:
agg = (
    df_all
    .groupby(["key", "source"])
    .agg(
        mean_quality=("quality", "mean"),
        std_quality=("quality", "std"),
        count=("quality", "count")
    )
    .reset_index()
)


In [39]:
pivot = agg.pivot(
    index="key",
    columns="source",
    values="mean_quality"
)


In [40]:
for col in pivot.columns:
    if col != "basic":
        pivot[f"delta_{col}"] = pivot[col] - pivot["basic"]


In [41]:
result = pivot.reset_index()
result


source,key,basic,learning_U10_10_A1_F1,learning_U10_10_A1_F3,learning_U10_10_A3_F3,learning_U10_5_5_A1_F1,learning_U10_5_5_A1_F3,learning_U10_5_5_A3_F3,learning_U10_5_A3_F3,learning_U5_5_A3_F3,delta_learning_U10_10_A1_F1,delta_learning_U10_10_A1_F3,delta_learning_U10_10_A3_F3,delta_learning_U10_5_5_A1_F1,delta_learning_U10_5_5_A1_F3,delta_learning_U10_5_5_A3_F3,delta_learning_U10_5_A3_F3,delta_learning_U5_5_A3_F3
0,10_100_2,8.6,8.7,8.7,8.6,8.600000,8.6,8.7,8.7,8.6,0.1,0.1,0.0,0.000000,0.0,0.1,0.1,0.0
1,10_100_4,6.2,6.2,6.3,6.2,6.200000,6.2,6.2,6.2,6.3,0.0,0.1,0.0,0.000000,0.0,0.0,0.0,0.1
2,10_200_2,10.3,10.3,10.3,10.3,10.300000,10.3,10.3,10.3,10.3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,10_200_4,6.2,6.1,6.1,6.1,6.100000,6.1,6.1,6.2,6.1,-0.1,-0.1,-0.1,-0.100000,-0.1,-0.1,0.0,-0.1
4,10_500_2,9.5,9.5,9.5,9.5,9.500000,9.6,9.5,9.5,9.5,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.0
5,10_500_4,6.1,6.1,6.1,6.1,6.100000,6.1,6.1,6.1,6.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,10_50_2,9.1,9.1,9.1,9.1,9.100000,9.1,9.1,9.1,9.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,10_50_4,6.0,6.0,6.1,6.1,6.000000,6.1,6.0,6.1,6.1,0.0,0.1,0.1,0.000000,0.1,0.0,0.1,0.1
8,2_100_2,72.8,72.7,72.7,72.4,72.700000,72.7,72.7,72.7,72.7,-0.1,-0.1,-0.4,-0.100000,-0.1,-0.1,-0.1,-0.1
9,2_100_4,61.7,61.7,61.7,60.0,61.700000,61.7,61.7,61.3,61.5,0.0,0.0,-1.7,0.000000,0.0,0.0,-0.4,-0.2


In [ ]:
result.to_csv("comparison_basic_vs_learning.csv", index=False)

In [42]:
result.drop(columns=["key"])mean()

TypeError: Could not convert ['10_100_210_100_410_200_210_200_410_500_210_500_410_50_210_50_42_100_22_100_42_200_22_200_42_500_22_500_42_50_22_50_43_100_23_100_43_200_23_200_43_500_23_500_43_50_23_50_45_100_25_100_45_200_25_200_45_500_25_500_45_50_25_50_4'] to numeric